In [15]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns',10)
pd.set_option('display.max_colwidth',10)
MYNA = np.nan
tchInfos = pd.read_csv('./data/tch2neo.csv',header=0,delimiter='\t',index_col='教师编号')
NODATA = '无'
tchInfos.fillna(NODATA, inplace=True)
COMMONINDEX = tchInfos.index

In [16]:
tchInfos.head()

,姓名,出生年月,性别,学历,学位,...,社会职务,荣誉及奖励,招生信息,导师资格,籍贯
教师编号,,,,,,,,,,,
0,安天庆,1963-0...,男,博士研究生,博士学位,...,江苏省数学会...,无,每年招收博士...,博士生导师,甘肃古浪
1,安杰晶,1988-1...,女,硕士研究生,硕士学位,...,无,无,无,无,无
2,卞培良,1993-0...,男,博士研究生,工学博士学位,...,无,无,无,无,无
3,边钰,1995-1...,女,硕士研究生,硕士学位,...,无,无,无,无,无
4,毕利东,1980-1...,男,博士研究生,博士学位,...,无,无,无,硕士生导师,无


### 个人简介中分离出籍贯

In [4]:
baseInfo = tchInfos['个人简介']

In [5]:
def GetNativePlace(baseInfo):
    if pd.isna(baseInfo):
        return MYNA
    pattern = '，(黑龙江|辽宁|吉林|河北|河南|湖北|湖南|山东|山西|陕西|安徽|浙江|江苏|福建|广东|海南|四川|云南|贵州|青海|甘肃|江西|台湾)(.{0,8})人，'
    NativePlaceList = re.findall(pattern,baseInfo)
    return MYNA if len(NativePlaceList)==0 else ''.join(NativePlaceList[0])

NativePlaces = pd.Series([GetNativePlace(baseInfoi) for baseInfoi in baseInfo],index=COMMONINDEX,name='籍贯')


### csv测试

In [6]:
import csv
# baseTriple = []
# for index in range(len(neoTestCsv)):
#     baseTriple.append({'id':index, nameColumn:neoTestCsv.iloc[index][nameColumn]})
# baseTriple

# neoTriple = []
# for index in range(len(neoTestCsv)):
#     neoTriple.append([(neoTestCsv.iloc[index][nameColumn],attri,neoTestCsv.iloc[index][attri]) for attri in attrColumns])
# neoTriple

# with open('./data/csvs/'+nameColumn+'.csv','w') as file:
#     teacherListHeader = ['id',nameColumn]
#     csvWriter = csv.DictWriter(file,teacherListHeader,delimiter=',',lineterminator='\n')
#     csvWriter.writeheader()
#     csvWriter.writerows(baseTriple)


### 写入neo4j

In [219]:
from py2neo import Graph,Node,Relationship,Subgraph
from py2neo.matching import NodeMatcher,RelationshipMatcher

In [220]:
neoGraph = Graph('bolt://localhost:7687/',auth=('neo4j','password'))

In [221]:
nMatcher = NodeMatcher(neoGraph)

In [2]:
nameColumn = '姓名'
attrColumns = ['出生年月', '性别', '学历', '学位' , '主页链接', '职称', '导师资格', '学科',
              '电话', '邮件', '办公地址', '通讯地址', '邮编', '籍贯']
otherColumns = ['头像链接']
analysisColumns = ['个人简介', '招生信息' ]
nodeColumns = ['相关教师', '毕业院校', '部门', '教学部门']
longColumns = ['教育经历', '工作经历', '科研项目', '论文', '科技成果',
              '开授课程', '教学成果','教学资源', '荣誉及奖励', '研究领域', '社会职务']


#### 工具代码

In [ ]:
neoGraph.run("CREATE CONSTRAINT ON (t:教师) ASSERT t.tchID IS UNIQUE")

In [ ]:
neoGraph.run("DROP CONSTRAINT ON (t:教师) ASSERT t.tchID IS UNIQUE")

In [ ]:
neoGraph.run("MATCH (n:教师) DELETE (n)")

In [ ]:
tx = neoGraph.begin()
# tx.create(Subgraph(nodes))
neoGraph.commit(tx)

#### 创建教师节点

In [ ]:
# 教师节点
nodes=[]
for index in range(len(tchInfos)):
    attrDict = tchInfos.iloc[index][attrColumns].to_dict()
    nodei = Node("教师", tchID=index, name=tchInfos.iloc[index][nameColumn], **attrDict)
    nodes.append(nodei)
len(nodes)

975

#### 教师间关联

In [203]:
relatedTchs = tchInfos['相关教师']

In [ ]:
relations = []
for index,tchi in enumerate(relatedTchs):
    firstNode = nMatcher.match('教师',tchID=index).first()
    relates = tchi.split(';')
    if relates[0]=='无':
        continue

    secondNodes = [nMatcher.match('教师',name=tchName).first() for tchName in relates]

    relations.extend([Relationship(firstNode,'相关',secondNode) for secondNode in secondNodes if secondNode!=None])

relations
    

In [ ]:
# tx = neoGraph.begin()
# tx.create(Subgraph(relationships=relations))
# neoGraph.commit(tx)

#### 毕业院校

In [146]:
graduateSchool = tchInfos['毕业院校']

In [ ]:
schoolSet = set(graduateSchool.to_list())
schoolNodes = [Node("学校", name=school) for school in schoolSet]
len(schoolNodes)
# tx = neoGraph.begin()
# tx.create(Subgraph(schoolNodes))
# neoGraph.commit(tx)

152

In [ ]:
schoolRelations = []
for tchi in range(len(graduateSchool)):
    firstNode = nMatcher.match('教师',tchID=tchi).first()
    secondNode = nMatcher.match('学校',name=graduateSchool.iloc[tchi]).first()
    schoolRelations.append(Relationship(firstNode,'毕业于',secondNode))
len(schoolRelations)
# tx = neoGraph.begin()
# tx.create(Subgraph(relationships=schoolRelations))
# neoGraph.commit(tx)

#### 籍贯

In [ ]:
for index,space in enumerate(tchInfos['籍贯']):
    tchi = nMatcher.match('教师',tchID=index).first()
    tchi['籍贯'] = space
    # neoGraph.push(tchi)

#### 部门, 教学部门

In [ ]:
departMent1 = tchInfos['部门']
departMent2 = tchInfos['教学部门']

In [ ]:
departs1 = set(sum([re.split('[、| |,|，]',d1i) for d1i in departMent1],[]))
departs1.remove('')
departs2 = set(sum([re.split('[、| |,|，]',d2i) for d2i in departMent2],[]))
departs2.remove('')
departsAll = set.union(departs1,departs2)
len(departs1),len(departs2),len(departsAll)

In [ ]:

departNodes = [Node("部门", name=depart) for depart in departsAll]
len(departNodes)
# tx = neoGraph.begin()
# tx.create(Subgraph(departNodes))
# neoGraph.commit(tx)

In [ ]:
departRelations = []
for tchi in range(len(departMent1)):
    firstNode = nMatcher.match('教师',tchID=tchi).first()
    departsi = set(re.split('[、| |,|，]',departMent1.iloc[tchi])+re.split('[、| |,|，]',departMent1.iloc[tchi]))
    if '信息学部' in departsi:
        departsi.remove('信息学部')
    if '' in departsi:
        departsi.remove('')
    secondNodes = [nMatcher.match('部门',name=departi).first() for departi in departsi]
    departRelations=[Relationship(firstNode,'属于',secondNode) for secondNode in secondNodes]
departRelations

In [ ]:
# tx = neoGraph.begin()
# tx.create(Subgraph(relationships=departRelations))
# neoGraph.commit(tx)

#### 教育经历

In [4]:
educations = tchInfos['教育经历']

In [169]:
# 添加学校教育经历中提到的学校

# extraSchoolSet = set()
# for index,education in enumerate(educations):
#     if education == NODATA:
#         continue
#     eduList = re.split('\r\n',education)
#     eduListTuple = [re.split('，',edui) for edui in eduList]
#     for eduTuple in eduListTuple:
#         if len(eduTuple) != 3 and len(eduTuple) != 4:
#             # print(index,eduTuple)
#             continue
#         schooli = eduTuple[1].strip()
#         if '大学' in schooli:
#             schooli = schooli[:schooli.find('大学')+2]
#         if schooli not in schoolSet:
#             print(index,schooli)
#             extraSchoolSet.add(schooli)


# removeList = ['2020/09/24','华东水利学院（河海大学','南京工学院','南京工学院（现东南大学','暖通空调','机械科学与工程','经济法',
# '舟山中学','定海二中','干览中心小学','水工结构工程','热能与动力工程','电力系统及其自动化','航空摄影测量及遥感专业',"大地测量学与测量工程"]
# for item in removeList:
#     extraSchoolSet.remove(item)

# len(extraSchoolSet)

# schoolNodesExtra = [Node("学校", name=school) for school in extraSchoolSet]

# tx = neoGraph.begin()
# tx.create(Subgraph(schoolNodesExtra))
# neoGraph.commit(tx)
# schoolSetAll =  list(set.union(schoolSet,extraSchoolSet))
# pd.Series(schoolSetAll,name='学校').to_csv('./data/csvs/学校.csv')


# schoolNodes = [Node("学校", name='武汉测绘学院'),Node("学校", name='University of Illinois at Urbana-Champaign')]
# print(schoolNodes)
# tx = neoGraph.begin()
# tx.create(Subgraph(schoolNodes))
# neoGraph.commit(tx)

# tx = neoGraph.begin()
# tx.create(Node("学校", name='剑桥大学'))
# neoGraph.commit(tx)

# schoolNodes = [Node("学校", name='美国沙漠研究所'),Node("学校", name='Queen Mary, University of London')]
# print(schoolNodes)
# tx = neoGraph.begin()
# tx.create(Subgraph(schoolNodes))
# neoGraph.commit(tx)

# tchi = nMatcher.match('学校',name='Queen Mary, University of London').first()
# tchi['name'] = 'Queen Mary，University of London'
# neoGraph.push(tchi)

# schoolNodes = [ Node("学校", name='University of Minnesota'),
#                 Node("学校", name='Temple University'),
#                 Node("学校", name='University of Nevada'),
#                 Node("学校", name='University of California'),
#                 Node("学校", name='法国南特大学'),
#                 ]
# tx = neoGraph.begin()
# tx.create(Subgraph(schoolNodes))
# neoGraph.commit(tx)

schoolNodes = [ Node("学校", name='University of Calgary'),
                Node("学校", name='中南大学'),]
tx = neoGraph.begin()
tx.create(Subgraph(schoolNodes))
neoGraph.commit(tx)

In [5]:
schoolSetSery = pd.read_csv('./data/csvs/学校.csv',index_col='id')['学校']
schoolSet = set(schoolSetSery.to_list())
degreeSet = set(['本科','学士',
                '硕士','研究生',
                '硕博','博士',
                '博士后',
                '访问','访学'])
degreeDict = {'本科':'学士','学士':'学士',
              '硕士':'硕士','研究生':'硕士',
                '硕博':'博士','博士':'博士',
                '博士后':'博士后',
                '访问':'访学','访学':'访学'}

In [10]:

def TupleLen2Fun(tupleLen):

    def SEduTuple(tchID,eduTuple):
        '''short length tuple'''
        school, degree, eduDays = None,None,None
        # schoolList = [schoolSeti for schoolSeti in schoolSet 
        #                           for tuplei in eduTuple if schoolSeti in tuplei]
        # degreeList = [degreeSeti for degreeSeti in degreeSet 
        #                           for tuplei in eduTuple if degreeSeti in tuplei]
        # if len(schoolList)!=0 and len(degreeList)!=0:
        #     school,degree = schoolList[0],degreeList[0]
        # print(tchID,eduTuple,': ',eduDays, school, degree)
        # print(tchID,eduTuple)
        return school, degree, eduDays

    def NEduTuple(tchID,eduTuple):
        '''normal length tuple'''
        school, degree, eduDays = eduTuple[1].strip(),eduTuple[-1],eduTuple[0].strip()

        if school not in schoolSet:
            schooliList = [schoolSeti for schoolSeti in schoolSet if schoolSeti in school]
            school = None if len(schooliList)==0 else schooliList[0]
        
        degreeList = [degreeSeti for degreeSeti in degreeSet if degreeSeti in degree]
        if len(degreeList)==1:
            degree = degreeList[0]
        elif len(degreeList)==2:
            degree = degreeList[1] if degreeList[0]=='研究生' else degreeList[0]
        else:
            degree =None

        eduDays = None if re.match('^\d{4}.*$',eduDays) is None else eduDays

        return school, degree, eduDays

    return SEduTuple if tupleLen<3 else NEduTuple

relationsTuple = []

for index,education in enumerate(educations):
    if education == NODATA:
        continue

    eduListTuple = [re.split('，',edui) for edui in re.split('\r\n',education)]

    for eduTuple in eduListTuple:
        school, degree, eduDays = TupleLen2Fun(len(eduTuple))(index,eduTuple)
        if school is None or degree is None or eduDays is None:
            continue
        relationsTuple.append((index,school,degreeDict[degree],eduDays))
        # relationsTuple.append((tchInfos.iloc[index]['姓名'],school,degreeDict[degree],eduDays))

In [11]:
len(relationsTuple)

1245

In [12]:
pd.DataFrame(relationsTuple,columns=['教师姓名', '大学', '学位', '时间'])

,教师姓名,大学,学位,时间
0,毕利东,北京林业大学,学士,1999.09-2003.07
1,毕利东,南京土壤研究所,博士,2003.09-2008.12
2,卞夏,东南大学,博士,2010.10-2014.11
3,卞夏,东南大学,硕士,2008.09-2010.09
4,卞夏,东南大学,学士,2004.09-2008.06
...,...,...,...,...
1240,张道锋,上海交通大学,博士,2013.9-2018.12
1241,赵振宙,东南大学,博士,2004.03-2007.11
1242,赵振宙,内蒙古工业大学,学士,1998.09-2002.07
1243,张犁,南京航空航天大学,博士,2007.09-2012.07


In [ ]:
neoRelations = []
for index,school,degree,eduDays in relationsTuple:
    firstNode = nMatcher.match('教师',tchID=index).first()
    secondNode = nMatcher.match('学校',name=school).first()
    neoRelations.append(Relationship(firstNode,degree,secondNode,time=eduDays)) 
neoRelations

In [230]:
# tx = neoGraph.begin()
# tx.create(Subgraph(relationships=neoRelations))
# neoGraph.commit(tx)